In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Large_B-cell_Lymphoma"
cohort = "GSE173263"

# Input paths
in_trait_dir = "../../input/GEO/Large_B-cell_Lymphoma"
in_cohort_dir = "../../input/GEO/Large_B-cell_Lymphoma/GSE173263"

# Output paths
out_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/GSE173263.csv"
out_gene_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE173263.csv"
out_clinical_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE173263.csv"
json_path = "../../output/preprocess/Large_B-cell_Lymphoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profile in DLBCL cases according to response to R-CHOP"
!Series_summary	"Gene expression profile (GEP) was analyzed in DLBCL cases to compare early failure patients (10) vs. responding patients (29), to identify features associated to primary chemoresistance"
!Series_overall_design	"We used frozen samples of DLBCL cases to analyze GEP in 2 groups: early failure (EF) cases (10), defined as refractory to induction treatment or early relapsing  (<12 months from diagnosis); and responding patients (29). Data was then analyzed with GSEA program"
Sample Characteristics Dictionary:
{0: ['tissue: Frozen lymph node biopsy'], 1: ['disease state: Diffuse large B-cell lymphoma (DLBCL)'], 2: ['response to r-chop: remission', 'response to r-chop: Early failure']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
is_gene_available = True  # Based on the Series title and summary, this dataset contains gene expression data

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 2  # "response to r-chop" is our trait of interest (remission vs. Early failure)
age_row = None  # Age data is not available in the sample characteristics
gender_row = None  # Gender data is not available in the sample characteristics

# 2.2 Data Type Conversion
def convert_trait(value):
    if ":" not in value:
        return None
    value = value.split(":", 1)[1].strip().lower()
    if "remission" in value:
        return 0  # Remission as control group
    elif "early failure" in value:
        return 1  # Early failure as case group
    else:
        return None

def convert_age(value):
    # Not used as age data is not available
    return None

def convert_gender(value):
    # Not used as gender data is not available
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_df)
    print("Clinical features preview:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical features preview:
{'GSM5264464': [0.0], 'GSM5264465': [0.0], 'GSM5264466': [0.0], 'GSM5264467': [0.0], 'GSM5264468': [0.0], 'GSM5264469': [0.0], 'GSM5264470': [0.0], 'GSM5264471': [0.0], 'GSM5264472': [0.0], 'GSM5264473': [0.0], 'GSM5264474': [0.0], 'GSM5264475': [0.0], 'GSM5264476': [0.0], 'GSM5264477': [0.0], 'GSM5264478': [0.0], 'GSM5264479': [0.0], 'GSM5264480': [0.0], 'GSM5264481': [0.0], 'GSM5264482': [0.0], 'GSM5264483': [0.0], 'GSM5264484': [0.0], 'GSM5264485': [0.0], 'GSM5264486': [0.0], 'GSM5264487': [0.0], 'GSM5264488': [0.0], 'GSM5264489': [0.0], 'GSM5264490': [0.0], 'GSM5264491': [0.0], 'GSM5264492': [0.0], 'GSM5264493': [1.0], 'GSM5264494': [1.0], 'GSM5264495': [1.0], 'GSM5264496': [1.0], 'GSM5264497': [1.0], 'GSM5264498': [1.0], 'GSM5264499': [1.0], 'GSM5264500': [1.0], 'GSM5264501': [1.0], 'GSM5264502': [1.0]}
Clinical data saved to ../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE173263.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Gene expression profile in DLBCL cases according to response to R-CHOP"
Line 1: !Series_geo_accession	"GSE173263"
Line 2: !Series_status	"Public on Apr 01 2022"
Line 3: !Series_submission_date	"Apr 25 2021"
Line 4: !Series_last_update_date	"Jul 01 2022"
Line 5: !Series_pubmed_id	"34632572"
Line 6: !Series_summary	"Gene expression profile (GEP) was analyzed in DLBCL cases to compare early failure patients (10) vs. responding patients (29), to identify features associated to primary chemoresistance"
Line 7: !Series_overall_design	"We used frozen samples of DLBCL cases to analyze GEP in 2 groups: early failure (EF) cases (10), defined as refractory to induction treatment or early relapsing  (<12 months from diagnosis); and responding patients (29). Data was then analyzed with GSEA program"
Line 8: !Series_type	"Expression profiling by array"
Line 9: !Series_contributor	"Ivan,,Dlouhy"
Found table marker at line 59
First few lines af

### Step 4: Gene Identifier Review

In [5]:
# Examine the gene identifiers
# The IDs follow a format like "11715100_at", "11715101_s_at", etc.
# These appear to be Affymetrix probe IDs, not human gene symbols
# The "_at", "_s_at", and "_x_at" suffixes are typical for Affymetrix arrays
# These will need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1967952 rows

Gene annotation preview (first few rows):
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['20-Aug-10', '20-Aug-10', '20-Aug-10', '20-Aug-10', '20-Aug-10'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Descrip

### Step 6: Gene Identifier Mapping

In [7]:
# Examining the gene annotation and expression data from previous steps
# The gene identifiers in gene_data are in the 'ID' column of gene_annotation
# The gene symbols are in the 'Gene Symbol' column of gene_annotation

# 1. Identify the mapping columns
print("Mapping Affymetrix probe IDs to human gene symbols...")
prob_col = 'ID'  # The column in gene_annotation containing probe IDs
gene_col = 'Gene Symbol'  # The column containing gene symbols

# 2. Create the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Created mapping between {len(mapping_df)} probes and gene symbols")
print("First few rows of mapping data:")
print(preview_df(mapping_df))

# 3. Apply the gene mapping to convert probe-level data to gene-level data
print("Converting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Successfully converted to gene expression data with {len(gene_data)} genes and {gene_data.shape[1]} samples")

# Preview the gene expression data
print("First few rows of gene expression data:")
print(preview_df(gene_data))

# Save the gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapping Affymetrix probe IDs to human gene symbols...
Created mapping between 49384 probes and gene symbols
First few rows of mapping data:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'Gene': ['HIST1H3G', 'HIST1H3G', 'HIST1H3G', 'TNFAIP8L1', 'OTOP2']}
Converting probe-level measurements to gene expression data...


Successfully converted to gene expression data with 19521 genes and 39 samples
First few rows of gene expression data:
{'GSM5264464': [6.408868, 8.506178, 41.900052, 6.349136, 10.87007], 'GSM5264465': [5.919338, 8.798117000000001, 41.460929, 5.356535, 10.3921], 'GSM5264466': [7.570321, 9.305402, 44.119149, 7.278757, 8.402752], 'GSM5264467': [6.092886, 8.964243, 42.475441, 5.828587, 9.705988], 'GSM5264468': [6.740193, 8.67219, 41.202211, 6.312686, 8.841656], 'GSM5264469': [8.684087, 9.200528, 44.634918, 6.898893, 8.51055], 'GSM5264470': [8.439163, 8.72709, 45.164675, 6.77206, 8.774792], 'GSM5264471': [7.746153, 9.36621, 41.89748, 5.936729, 10.66011], 'GSM5264472': [7.088106, 8.873732, 40.881253, 6.329117, 10.44842], 'GSM5264473': [6.40157, 8.719936, 42.478854, 6.974187, 9.601769], 'GSM5264474': [6.455291, 9.706564, 44.217554, 5.618299, 9.159767], 'GSM5264475': [5.635584, 8.32609, 40.963491, 5.865566, 9.813807], 'GSM5264476': [5.961028, 8.43795, 41.006274, 5.796535, 10.68841], 'GSM526447

Gene expression data saved to ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE173263.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print("\nNormalizing gene symbols...")
# Get the gene data from previous step if needed
try:
    # First, check if we need to reload the gene data
    gene_data_path = out_gene_data_file  # Use the correct variable from setup
    if os.path.exists(gene_data_path):
        print("Loading gene data from previous step...")
        gene_data = pd.read_csv(gene_data_path, index_col=0)
    else:
        print("Gene data not found, using the gene_data from previous processing...")
    
    print(f"Gene data shape before normalization: {gene_data.shape}")
    print("Sample of gene symbols before normalization:", gene_data.index[:10].tolist())
    
    # Use the normalize_gene_symbols_in_index function to standardize gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(normalized_gene_data.index)} genes")
    
    # Handle case where normalization results in 0 genes
    if len(normalized_gene_data.index) == 0:
        print("WARNING: Normalization resulted in 0 genes. Using original gene data.")
        normalized_gene_data = gene_data  # Use original data
        is_gene_available = False  # Mark that normalization failed
    else:
        is_gene_available = True
    
    # Save the normalized gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error in gene normalization: {str(e)}")
    is_gene_available = False

# 2. Load clinical data
print("\nLoading clinical data...")
try:
    if os.path.exists(out_clinical_data_file):
        clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
        print(f"Loaded clinical data with shape: {clinical_df.shape}")
        print(f"Clinical data columns: {clinical_df.columns.tolist()}")
        is_trait_available = True
    else:
        print("Clinical data file not found.")
        is_trait_available = False
except Exception as e:
    print(f"Error loading clinical data: {str(e)}")
    is_trait_available = False

# 3. Link clinical and genetic data if both are available
if is_gene_available and is_trait_available:
    print("\nLinking clinical and genetic data...")
    try:
        # Ensure clinical data has a proper DataFrame structure
        if clinical_df.shape[0] == 1:  # If it's a single row (trait)
            clinical_df = clinical_df.T  # Transpose if needed
            
        print(f"Clinical data shape after adjustment: {clinical_df.shape}")
        
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print(f"Linked data columns: {linked_data.columns[:5].tolist()}")  # Show first few columns
        
        # Identify the trait column name (should be the first column in clinical_df)
        trait_col = clinical_df.index[0]  # Get the trait name from the index
        print(f"Using trait column: {trait_col}")
        
        # 4. Handle missing values in the linked data
        print("\nHandling missing values...")
        linked_data = handle_missing_values(linked_data, trait_col)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # 5. Check for biased features
        print("\nChecking for biased features...")
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait_col)
        
    except Exception as e:
        print(f"Error in data linking or processing: {str(e)}")
        import traceback
        traceback.print_exc()
        is_trait_available = False
        is_biased = True
        linked_data = pd.DataFrame()  # Empty DataFrame as placeholder
else:
    print("\nCannot link data: either gene data or clinical data is unavailable")
    is_biased = True
    linked_data = pd.DataFrame()  # Empty DataFrame as placeholder

# 6. Validate and save cohort information
print("\nPerforming final validation...")
note = ""
if not is_gene_available:
    note = "Gene expression data normalization failed."
elif not is_trait_available:
    note = "Clinical trait data is not available or could not be loaded."
elif is_biased:
    note = "The distribution of the trait in this dataset is severely biased."
else:
    note = "Dataset contains usable gene expression data and treatment response information for DLBCL patients."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    print(f"\nSaving linked data to {out_data_file}...")
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved successfully to {out_data_file}")
else:
    print(f"\nData not saved as it was determined to be unusable")

# 8. Report final status
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if note:
    print(f"Note: {note}")


Normalizing gene symbols...
Loading gene data from previous step...
Gene data shape before normalization: (19521, 39)
Sample of gene symbols before normalization: ['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1']
After normalization: 19298 genes


Normalized gene data saved to ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE173263.csv

Loading clinical data...
Loaded clinical data with shape: (1, 39)
Clinical data columns: ['GSM5264464', 'GSM5264465', 'GSM5264466', 'GSM5264467', 'GSM5264468', 'GSM5264469', 'GSM5264470', 'GSM5264471', 'GSM5264472', 'GSM5264473', 'GSM5264474', 'GSM5264475', 'GSM5264476', 'GSM5264477', 'GSM5264478', 'GSM5264479', 'GSM5264480', 'GSM5264481', 'GSM5264482', 'GSM5264483', 'GSM5264484', 'GSM5264485', 'GSM5264486', 'GSM5264487', 'GSM5264488', 'GSM5264489', 'GSM5264490', 'GSM5264491', 'GSM5264492', 'GSM5264493', 'GSM5264494', 'GSM5264495', 'GSM5264496', 'GSM5264497', 'GSM5264498', 'GSM5264499', 'GSM5264500', 'GSM5264501', 'GSM5264502']

Linking clinical and genetic data...
Clinical data shape after adjustment: (39, 1)
Linked data shape: (40, 19337)
Linked data columns: ['GSM5264464', 'GSM5264465', 'GSM5264466', 'GSM5264467', 'GSM5264468']
Using trait column: GSM5264464

Handling missing values.